In [87]:
import pymongo
from dotenv import load_dotenv
import os
import itertools
import haversine as hs
import json
import time
from math import pi, sqrt, cos, prod

load_dotenv()

mongodb_uri = os.environ.get("MONGODB_URI")
client = pymongo.MongoClient(mongodb_uri)
db = client["schedule"]
classes = db["classes"]
locations = db["locations"]

In [88]:
weights = {
    "distance": 40,
    "back_to_back": 10,
    "time": 50,
    "section": 30,
    "lunch": 20
}

class ScheduleException(Exception):
    pass

class ScheduleOverlapException(Exception):
    # When a schedule is not possible because 2 classes overlap time-wise
    pass

In [89]:
def check_for_no_section(classes_to_take, open_sections_filter_applied):
    for clas in classes_to_take:
        if len(clas["sections"]) == 0:
            if (open_sections_filter_applied):
                raise ScheduleException(f"No open sections found for {clas['code']} {clas['number']} in your specified time range. Please try again with a different time range.")
            else:
                raise ScheduleException(f"No sections found for {clas['code']} {clas['number']} in your specified time range. Please try again with a different time range.")


def fetch_section_data(class_list):
    """
    Fetches the section data from the database
    """
    for class_ in class_list:
        clas_mongo_object = classes.find_one({"code": class_["code"], "number": class_[
                                             "number"], "year": 2024, "semester": "spring"})
        class_["sections"] = []
        for section in clas_mongo_object["sections"]:
            if section["crn"] in class_["crn_list"]:
                section["class"] = {
                    "code": class_["code"],
                    "number": class_["number"]
                }
                # Drop online lecture sections
                meeting_type_codes = [meeting["type_code"] for meeting in section["meetings"]]
                if ("OLC" in meeting_type_codes):
                    continue
                class_["sections"].append(section)

    return class_list


def is_preferred_time(section, start_time, end_time):
    earliest_meeting = start_time
    latest_meeting = end_time

    for meeting in section["meetings"]:
        if meeting['start_time'] is None or meeting['end_time'] is None:
            continue
        if meeting['start_time'].hour < earliest_meeting:
            earliest_meeting = meeting['start_time'].hour
        if meeting['end_time'].hour > latest_meeting:
            latest_meeting = meeting['end_time'].hour

    return earliest_meeting >= start_time and latest_meeting <= end_time


def delete_unpreferred_sections(class_list, start_time, end_time):
    for clas in class_list:
        clas["sections"] = [section for section in clas['sections'] if is_preferred_time(section, start_time, end_time)]
    return class_list


def delete_closed_sections(class_list, open_sections_only):
    if open_sections_only:
        for clas in class_list:
            clas["sections"] = [section for section in clas['sections']
                                if section['enrollment_status'].lower() != 'closed']
    return class_list


def apply_hard_filters(user_preferences):
    class_list = fetch_section_data(user_preferences["classes"])
    class_list = delete_unpreferred_sections(class_list, user_preferences["start_time"], user_preferences["end_time"])
    check_for_no_section(class_list, False)  # Check if any class has no sections

    class_list = delete_closed_sections(class_list, user_preferences["open_sections_only"])

    check_for_no_section(class_list, True)  # Check if any class has no sections

    return class_list


def remove_duplicates(grouped_sections, pref_sections):
    hash_list = set()
    new_list = []
    for section in grouped_sections:
        section_hash = f"{section['meetings'][0]['coordinates']}_{section['meetings'][0]['start_time']}_{section['meetings'][0]['end_time']}_{section['meetings'][0]['days']}"
        if section_hash not in hash_list:
            hash_list.add(section_hash)
            new_list.append(section)
        elif section['crn'] in pref_sections:
            new_list.append(section)

    return new_list


def generate_schedule_combinations(class_list, user_preferences):
    """
    Generates all possible schedules
    """
    groups = {}

    # Split classes into groups based on meeting types
    for clas in class_list:
        base_group_name = f"{clas['code']}_{clas['number']}_"
        for section in clas['sections']:
            list_of_meeting_types = [meeting['type_code']
                                     for meeting in section['meetings']]
            group_key = base_group_name + \
                "_".join([str(x) for x in list_of_meeting_types])
            groups[group_key] = groups.get(group_key, []) + [section]

    un_duplicated_groups = {}


    # Remove groups with section which have the same days, times, and locations
    for key, value in groups.items():
        un_duplicated_groups[key] = remove_duplicates(value, user_preferences["pref_sections"])

    possible_schedules_len = prod([len(x) for x in un_duplicated_groups.values()])
    possible_schedules = itertools.product(*un_duplicated_groups.values())
    print(f"Possible schedules: {possible_schedules_len}")
    return possible_schedules

## Generate Hashmaps

In [90]:
from linetimer import CodeTimer
user_prefs = {"classes": [{"code": "ECE", "number": "110", "crn_list": ["32463", "32460", "52912", "32470", "52914", "52910", "32466", "32461", "52913", "32456", "52911", "63640", "32459", "32464", "32471", "52909", "61723", "57693"]}, {"code": "ECE", "number": "220", "crn_list": ["61635", "61638", "61640", "61641", "61642", "61643", "64369", "61629", "63649", "63650"]}, {"code": "PHYS", "number": "212", "crn_list": ["38029", "38032", "57914", "56034", "58663", "51211", "58664", "53105", "38051", "38057", "38104", "38059", "58662", "57937", "38062",
                                                                                                                                                                                                                                                                                                                                                                                                                                                                        "57963", "58985", "58987", "58988", "58978", "58979"]}, {"code": "MATH", "number": "241", "crn_list": ["50318", "46053", "46054", "46056", "46057", "46058", "48355", "46061", "46062", "46065", "46066", "46063", "46064", "59561", "58071", "46060", "46067", "46059", "52979", "46070", "46071", "46072", "46073", "47543", "67808", "50322", "55923", "55924", "55925", "55922", "55926", "56027", "56026", "55921", "46074"]}, {"code": "CWL", "number": "207", "crn_list": ["65011"]}], "pref_sections": ["52914", "63640", "32471", "61643", "61642", "38096", "38088", "50318"], "classes_1": ["ECE 110", "ECE 220", "PHYS 212", "MATH 241", "CWL 207"], "open_sections_only": False, "start_time": 10, "end_time": 21, "pref_time": 16, "lunch": {"start": 11, "end": 14, "duration": 1}, "max_distance": 800, "back_to_back": True}
class_list = apply_hard_filters(user_prefs)
possible_schedules = generate_schedule_combinations(class_list, user_prefs)


Possible schedules: 4976640


## Initial Version

In [91]:
def convert_to_time_based(list_of_sections):
    schedule_slots = {
        'W': [None]*23,
        'R': [None]*23,
        'F': [None]*23,
        'T': [None]*23,
        'M': [None]*23,
    }

    for section in list_of_sections:
        for meeting in section['meetings']:

            if meeting['start_time'] is None or meeting['end_time'] is None:
                continue

            start_hour = meeting['start_time'].hour
            end_hour = meeting['end_time'].hour

            for day in meeting['days'].strip():
                for hour in range(start_hour, end_hour+1):

                    if schedule_slots[day][hour] is not None:
                        raise ScheduleOverlapException(
                            f"Schedule overlap between {schedule_slots[day][hour]['class']['code']} {schedule_slots[day][hour]['class']['number']} and {section['class']['code']} {section['class']['number']}")
                    schedule_slots[day][hour] = section

    return schedule_slots

In [92]:
with CodeTimer():
    for schedule in possible_schedules:
        try:
            convert_to_time_based(schedule)
        except ScheduleOverlapException:
            continue

Code block took: 22627.08649 ms


## Hashmap of Times and Days

In [93]:
def hash_schedules(class_list) :
    hashmap = {}

    for clas in class_list:
        for section in clas['sections']:
            for meeting in section['meetings']:
                if meeting['start_time'] is None or meeting['end_time'] is None:
                    continue
                start_hour = meeting['start_time'].hour
                end_hour = meeting['end_time'].hour
                section_dict = {}
                for day in list(meeting['days'].strip()):
                    section_dict[day] = (start_hour, end_hour)
                hashmap[section['crn']] = section_dict
    
    return hashmap

In [94]:
def convert_to_time_2(list_of_sections, schedule_time_hashmap) :
    schedule_slots = {
        'W': [None]*23,
        'R': [None]*23,
        'F': [None]*23,
        'T': [None]*23,
        'M': [None]*23,
    }

    for section in list_of_sections:
        for day in schedule_time_hashmap[section['crn']].keys() :
            start_hour = schedule_time_hashmap[section['crn']][day][0]
            end_hour = schedule_time_hashmap[section['crn']][day][1]
            for hour in range(start_hour, end_hour+1):
                section_copy = section
                if schedule_slots[day][hour] is not None:
                    raise ScheduleOverlapException(
                        f"Schedule overlap between {schedule_slots[day][hour]['class']['code']} {schedule_slots[day][hour]['class']['number']} and {section_copy['class']['code']} {section_copy['class']['number']}")
                schedule_slots[day][hour] = section_copy

    return schedule_slots

In [95]:
with CodeTimer():
    schedule_time_hashmap = hash_schedules(class_list)
    for schedule in possible_schedules:
        try:
            convert_to_time_2(schedule, schedule_time_hashmap)
        except ScheduleOverlapException:
            continue

Code block took: 0.15866 ms
